# COVID-19 Projection using LSTM

## Importing Necessary Libraries

In [64]:
# Import necessary libraries
import math
import pickle
import os
import pandas as pd
import folium 
import numpy as np
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pylab as plt
import seaborn as sns
from matplotlib import rcParams
import plotly as py
import cufflinks
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
from tqdm import tqdm_notebook as tqdm
import warnings
import tensorflow as tf
from numpy import array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Input
from tensorflow.keras.layers import BatchNormalization
from dateutil.relativedelta import relativedelta
import datetime
warnings.filterwarnings("ignore")


## Reading Data

In [65]:
# Reading COVID-19 Raw data
train = pd.read_csv("./covid19-global-forecasting-week-3/train.csv")
submission = pd.read_csv("./covid19-global-forecasting-week-3/submission.csv")
test = pd.read_csv("./covid19-global-forecasting-week-3/test.csv")

In [66]:
train.isna().sum()

Id                    0
Province_State    13552
Country_Region        0
Date                  0
ConfirmedCases        0
Fatalities            0
dtype: int64

## Cleaning Data

In [67]:
# filling the missing states with a value 'NoState'
train=train.fillna('NoState')
test=test.fillna('NoState')
# changing the data type
train=train.rename(columns={'ConfirmedCases':'Confirmed','Fatalities':'Deaths','Country_Region':'Country/Region',
                     'Province_State':'Province/State','Date':'ObservationDate'})
num_cols=['Confirmed', 'Deaths']
for col in num_cols:
    temp=[int(i) for i in train[col]]
    train[col]=temp 
train.head(2)

,Id,Province/State,Country/Region,ObservationDate,Confirmed,Deaths
0,1,NoState,Afghanistan,2020-01-22,0,0
1,2,NoState,Afghanistan,2020-01-23,0,0


Let's prepare a list of all states of each region/country

In [68]:
# Creating list of all regions of all countries
unique_regions=train['Country/Region'].unique()
states_per_regions=[]
for reg in tqdm(unique_regions):
    states_per_regions.append(train[train['Country/Region']==reg]['Province/State'].unique()) 
print('No of unique regions:',len(unique_regions))    


No of unique regions: 180


## Data Preprocessing

In [69]:
# function to create training data for LSTM
def create_train_dataset(target,n_steps,train,pivot_date):
    train = train.query("ObservationDate<"+pivot_date)
    x=[]
    y=[]
    for k in tqdm(range(len(unique_regions))):
        for state in states_per_regions[k]:
            #print(unique_regions[k],state)
            temp=train[(train['Country/Region']==unique_regions[k]) &(train['Province/State']==state)]
            sequence=list(temp[target])
            for i in range(len(sequence)):
                end_ix = i + n_steps
                if end_ix > len(sequence)-1:
                    break
                seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
                if(seq_y!=0):
                    x.append(seq_x)   
                    y.append(seq_y)
    return array(x),array(y)    

def create_countrywise_newly_added_train_dataset(target,n_steps,train,pivot_date):
    train = train.query("ObservationDate<"+pivot_date)
    x=[]
    y=[]
    for k in tqdm(range(len(unique_regions))):
            #print(unique_regions[k],state)
        temp=train[(train['Country/Region']==unique_regions[k])]
        sequence=list(temp[target])
        for i in range(len(sequence)):
            end_ix = i + n_steps
            if end_ix > len(sequence)-1:
                break
            seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
            if(seq_y!=0):
                x.append(seq_x)   
                y.append(seq_y)
    return array(x),array(y)  

In [70]:
# function to create test dataset
# target : 'Confirmed'/'Deaths'
def create_test_dataset(target,n_steps,train,pivot_date):
    train = train.query("ObservationDate<"+pivot_date)
    x=[]
    regs=[]
    for k in tqdm(range(len(unique_regions))):
        for state in states_per_regions[k]:
            #regs.append((unique_regions[k],state))
            temp=train[(train['Country/Region']==unique_regions[k]) &(train['Province/State']==state)]
            sequence=temp[target]
            #print(sequence[len(sequence)-n_steps:len(sequence)+1])
            x.append(sequence[len(sequence)-n_steps:len(sequence)+1])
            regs.append((unique_regions[k],state))      
    return array(x),regs

def create_countrywise_newly_added_test_dataset(target,n_steps,train,pivot_date):
    train = train.query("ObservationDate<"+pivot_date)
    x=[]
    regs=[]
    for k in tqdm(range(len(unique_regions))):
        temp=train[(train['Country/Region']==unique_regions[k])]
        sequence=temp[target]
        #print(sequence[len(sequence)-n_steps:len(sequence)+1])
        x.append(sequence[len(sequence)-n_steps:len(sequence)+1])
        regs.append(unique_regions[k])      
    return array(x),regs


In [71]:
def get_newly_added(world_data_):
    world_data_=world_data_.sort_values(['Country/Region','ObservationDate'])
    temp=[0*i for i in range(len(world_data_))]
    world_data_['New Confirmed']=temp
    world_data_['New Death']=temp
    for i in tqdm(range(1,len(world_data_))):
        if(world_data_['Country/Region'].iloc[i]==world_data_['Country/Region'].iloc[i-1]):
            if(world_data_['Deaths'].iloc[i]<world_data_['Deaths'].iloc[i-1]):
                world_data_['Deaths'].iloc[i]=world_data_['Deaths'].iloc[i-1]
            if(world_data_['Confirmed'].iloc[i]<world_data_['Confirmed'].iloc[i-1]):
                world_data_['Confirmed'].iloc[i]=world_data_['Confirmed'].iloc[i-1]     
            world_data_['New Confirmed'].iloc[i]=world_data_['Confirmed'].iloc[i]-world_data_['Confirmed'].iloc[i-1]
            world_data_['New Death'].iloc[i]=world_data_['Deaths'].iloc[i]-world_data_['Deaths'].iloc[i-1]
        else:
            world_data_['New Confirmed'].iloc[i]=world_data_['Confirmed'].iloc[i]
            world_data_['New Death'].iloc[i]=world_data_['Deaths'].iloc[i]
    return world_data_

In [72]:
# Countrywise timeseries data with Newly added Incident Each Day
covid_timeseries = train.groupby(['ObservationDate','Country/Region','Province/State'])['Confirmed', 'Deaths'].sum()
covid_timeseries=covid_timeseries.reset_index().sort_values('ObservationDate')
covid_timeseries=get_newly_added(covid_timeseries)
covid_timeseries[covid_timeseries['Country/Region']=='India'].tail()

,ObservationDate,Country/Region,Province/State,Confirmed,Deaths,New Confirmed,New Death
22171,2020-04-03,India,NoState,2567,72,24,0
22477,2020-04-04,India,NoState,3082,86,515,14
22783,2020-04-05,India,NoState,3588,99,506,13
23089,2020-04-06,India,NoState,4778,136,1190,37
23395,2020-04-07,India,NoState,5311,150,533,14


In [73]:
# Maintain the date format for pivot_date and forcast_start_date
# Pivot_date : data of date less than the given date will be used for training
# Forcast_start_date : Date from which forcasting will be started
n_steps=7
pivot_date="'2020-04-02'"
forcast_start_date='2020-04-02'
print('Preparing datasets with Cumulative Confirmed Incidents..')
X_c,y_c=create_train_dataset('Confirmed',n_steps,train,pivot_date)
print('Preparing datasets with Newly Confirmed Incidents..')
X_nc,y_nc=create_train_dataset('New Confirmed',n_steps,covid_timeseries,pivot_date)
test_confirmed,regs= create_test_dataset('Confirmed',n_steps,train,pivot_date)
test_nc,reg_nc=create_test_dataset('New Confirmed',n_steps,covid_timeseries,pivot_date)
print('Preparing datasets with Deaths Incidents..')
X_d,y_d=create_train_dataset('Deaths',n_steps,train,pivot_date)
test_deaths,regs= create_test_dataset('Deaths',n_steps,train,pivot_date)
print('Datasets prepared sucessfully.')

Preparing datasets with Cumulative Confirmed Incidents..



Preparing datasets with Newly Confirmed Incidents..



Preparing datasets with Deaths Incidents..



Datasets prepared sucessfully.


### Spliting the data

In [74]:
# Split the train data in to train and val data

X_train_c, X_val_c, y_train_c, y_val_c = train_test_split(X_c, y_c, test_size=0.30, random_state=42)
X_train_d, X_val_d, y_train_d, y_val_d = train_test_split(X_d, y_d, test_size=0.30, random_state=42)

X_train_nc, X_val_nc, y_train_nc, y_val_nc = train_test_split(X_c, y_c, test_size=0.30, random_state=42)


### Reshapping the data

In [75]:
# Reshapping the Confirmed data for LSTM
X_train_c = X_train_c.reshape((X_train_c.shape[0], 1, X_train_c.shape[1]))
X_val_c= X_val_c.reshape(( X_val_c.shape[0], 1,  X_val_c.shape[1]))
X_train_nc = X_train_nc.reshape((X_train_nc.shape[0], 1, X_train_nc.shape[1]))
X_val_nc= X_val_nc.reshape(( X_val_nc.shape[0], 1,  X_val_nc.shape[1]))
X_test_c= test_confirmed.reshape(( test_confirmed.shape[0], 1, test_confirmed.shape[1]))
X_test_nc= test_nc.reshape(( test_nc.shape[0], 1, test_nc.shape[1]))
print(X_train_c.shape, y_train_c.shape, X_val_c.shape, y_val_c.shape,X_test_c.shape,X_test_nc.shape)

(6636, 1, 7) (6636,) (2845, 1, 7) (2845,) (306, 1, 7) (306, 1, 7)


In [76]:
# Reshapping the Death data for LSTM
X_train_d = X_train_d.reshape((X_train_d.shape[0], 1, X_train_d.shape[1]))
X_val_d= X_val_d.reshape(( X_val_d.shape[0], 1,  X_val_d.shape[1]))
X_test_d= test_deaths.reshape(( test_deaths.shape[0], 1, test_deaths.shape[1]))
print(X_train_d.shape, y_train_d.shape, X_val_d.shape, y_val_d.shape,X_test_d.shape)

(3021, 1, 7) (3021,) (1295, 1, 7) (1295,) (306, 1, 7)


In [77]:
print(X_train_c[100])
print(X_train_d[1])

[[42 54 59 65 69 78 80]]
[[1 1 3 6 6 8 8]]


## Preparing the Model

In [78]:
# Initializing model components
epochs = 10
batch_size = 32
n_hidden = 32
timesteps = X_train_c.shape[1]
input_dim = X_train_c.shape[2]
n_features=1

print(timesteps)
print(input_dim)
print(len(X_train_c))

1
7
6636


### Stacked LSTM 

### Model for Confirmed cases

In [79]:
# Stacked LSTM Model
model_c = Sequential()
model_c.add(LSTM(50, activation='relu', input_shape=(n_features,n_steps),return_sequences=True))
model_c.add(LSTM(150, activation='relu'))
model_c.add(Dense(1,activation='relu'))
model_c.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 1, 50)             11600     
_________________________________________________________________
lstm_7 (LSTM)                (None, 150)               120600    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 151       
Total params: 132,351
Trainable params: 132,351
Non-trainable params: 0
_________________________________________________________________


In [80]:
# Compiling the model
model_c.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredLogarithmicError())
callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1, factor=0.6),
             EarlyStopping(monitor='val_loss', patience=20),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]
# fit the model
hist=model_c.fit(X_train_c,y_train_c, epochs=epochs, batch_size=batch_size, validation_data=(X_val_c, y_val_c), verbose=2, 
               shuffle=True,callbacks=callbacks)


Train on 6636 samples, validate on 2845 samples
Epoch 1/10
6636/6636 - 4s - loss: 1.1894 - val_loss: 0.0608
Epoch 2/10
6636/6636 - 1s - loss: 0.0622 - val_loss: 0.0524
Epoch 3/10
6636/6636 - 1s - loss: 0.0583 - val_loss: 0.0521
Epoch 4/10
6636/6636 - 1s - loss: 0.0569 - val_loss: 0.0502
Epoch 5/10
6636/6636 - 1s - loss: 0.0561 - val_loss: 0.0498
Epoch 6/10
6636/6636 - 1s - loss: 0.0555 - val_loss: 0.0502
Epoch 7/10
6636/6636 - 1s - loss: 0.0547 - val_loss: 0.0489
Epoch 8/10
6636/6636 - 1s - loss: 0.0542 - val_loss: 0.0488
Epoch 9/10
6636/6636 - 1s - loss: 0.0537 - val_loss: 0.0472
Epoch 10/10
6636/6636 - 1s - loss: 0.0537 - val_loss: 0.0486


In [81]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Epoch vs Loss for Confirmed Cases')
plt.ylabel('Loss')

plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='best')
plt.show()

<IPython.core.display.Javascript object>

### Model for Death Cases

In [82]:
# Stacked LSTM Model
model_d = Sequential()
model_d.add(LSTM(50, activation='relu', input_shape=(n_features,n_steps),return_sequences=True))
model_d.add(LSTM(50, activation='relu'))
model_d.add(Dense(1))
model_d.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 1, 50)             11600     
_________________________________________________________________
lstm_9 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 31,851
Trainable params: 31,851
Non-trainable params: 0
_________________________________________________________________


In [83]:
# Compiling the model
model_d.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredLogarithmicError())
callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1, factor=0.6),
             EarlyStopping(monitor='val_loss', patience=20),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]
# fit the model
hist=model_d.fit(X_train_d,y_train_d, epochs=epochs, batch_size=batch_size, validation_data=(X_val_d, y_val_d), verbose=2, 
               shuffle=True,callbacks=callbacks)


Train on 3021 samples, validate on 1295 samples
Epoch 1/10
3021/3021 - 3s - loss: 1.3756 - val_loss: 0.1665
Epoch 2/10
3021/3021 - 1s - loss: 0.0796 - val_loss: 0.0368
Epoch 3/10
3021/3021 - 1s - loss: 0.0368 - val_loss: 0.0281
Epoch 4/10
3021/3021 - 1s - loss: 0.0320 - val_loss: 0.0261
Epoch 5/10
3021/3021 - 0s - loss: 0.0305 - val_loss: 0.0263
Epoch 6/10
3021/3021 - 1s - loss: 0.0296 - val_loss: 0.0250
Epoch 7/10
3021/3021 - 0s - loss: 0.0288 - val_loss: 0.0253
Epoch 8/10
3021/3021 - 0s - loss: 0.0286 - val_loss: 0.0254
Epoch 9/10
3021/3021 - 1s - loss: 0.0285 - val_loss: 0.0247
Epoch 10/10
3021/3021 - 0s - loss: 0.0279 - val_loss: 0.0246


In [84]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Epoch vs Loss for Deathh Cases')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='best')
plt.show()

## Model to predict Newly Confirmed Cases

In [85]:
# Stacked LSTM Model
model_nc = Sequential()
model_nc.add(LSTM(50, activation='relu', input_shape=(n_features,n_steps),return_sequences=True))
model_nc.add(LSTM(50, activation='relu'))
model_nc.add(Dense(1))
model_nc.summary()
model_nc.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredLogarithmicError())
callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1, factor=0.6),
             EarlyStopping(monitor='val_loss', patience=20),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]
# fit the model
hist=model_nc.fit(X_train_nc,y_train_nc, epochs=epochs, batch_size=batch_size, validation_data=(X_val_nc, y_val_nc), verbose=2, 
               shuffle=True,callbacks=callbacks)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 1, 50)             11600     
_________________________________________________________________
lstm_11 (LSTM)               (None, 50)                20200     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 51        
Total params: 31,851
Trainable params: 31,851
Non-trainable params: 0
_________________________________________________________________
Train on 6636 samples, validate on 2845 samples
Epoch 1/10
6636/6636 - 4s - loss: 0.9941 - val_loss: 0.0710
Epoch 2/10
6636/6636 - 1s - loss: 0.0659 - val_loss: 0.0534
Epoch 3/10
6636/6636 - 1s - loss: 0.0590 - val_loss: 0.0505
Epoch 4/10
6636/6636 - 1s - loss: 0.0567 - val_loss: 0.0493
Epoch 5/10
6636/6636 - 1s - loss: 0.0553 - val_loss: 0.0487
Epoch 6/10
6636/6636 - 1s - loss

## Utilities for Prediction and Submission

In [86]:
import math
def pred(model,data):
    y_pred=model.predict(data)
    #y_pred=[math.ceil(i) for i in y_pred]
    return y_pred

In [87]:
# Utility method for Forcasting
# model - trained model on Confirmed/Deaths data
# start_date - Starting date of forcasting
# num_days - Number of days for which forcasting is required
def forcast(model,data,start_date,num_days):
    res_=dict()
    for i in range(len(data)):
        res_[i]=[]
    y_pred=pred(model,data)
    dates=[]
    date1 = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    for j in range(1,num_days+1):
        for i in range(len(data)):
            cur_window=list(data[i][0][1:n_steps+1])
            #print(j,i,cur_window[-1])
            res_[i].append(cur_window[-1])
            cur_window.append(y_pred[i])
            data[i][0]=cur_window
        y_pred=pred(model,data)
        dates.append(date1.strftime("%Y-%m-%d"))
        date1+=relativedelta(days=1)
    res=pd.DataFrame(pd.DataFrame(pd.DataFrame(res_).values.T)) 
    res.columns=dates
    res['Country/State']=regs
    return res

def forcast_(model,data,start_date,num_days):
    res_=[]
    for i in list(data['Country/Region']):
        res_.append(i)
    y_pred=pred(model,data)
    dates=[]
    date1 = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    for j in range(1,num_days+1):
        for i in range(len(data)):
            cur_window=list(data[i][0][1:n_steps+1])
            #print(j,i,cur_window[-1])
            res_[i].append(cur_window[-1])
            cur_window.append(y_pred[i])
            data[i][0]=cur_window
        y_pred=pred(model,data)
        dates.append(date1.strftime("%Y-%m-%d"))
        date1+=relativedelta(days=1)
    res=pd.DataFrame(pd.DataFrame(pd.DataFrame(res_).values.T)) 
    res.columns=dates
    res['Country/State']=res_
    return res
        

In [88]:
# Utility method for submission
def prepare_submission(res_c,res_d,res_nc,test,pivot_date):
    test=test.query("Date>="+pivot_date)
    index=dict()
    for i in range(len(res_c)):
        index[res_c.iloc[i]['Country/State']]=i
    pred_c=[]
    pred_d=[]
    pred_nc=[]
    for i in tqdm(range(len(test))):
        if((test.iloc[i]['Country_Region'],test.iloc[i]['Province_State']) in index):
            loc=index[(test.iloc[i]['Country_Region'],test.iloc[i]['Province_State'])]
            #print(res.iloc[loc][test.iloc[i]['Date']])
            pred_c.append(res_c.iloc[loc][test.iloc[i]['Date']])     
            pred_d.append(res_d.iloc[loc][test.iloc[i]['Date']]) 
            pred_nc.append(res_nc.iloc[loc][test.iloc[i]['Date']]) 
    test['ConfirmedCases']=pred_c
    test['Fatalities']=pred_d
    test['New Confirmed']=pred_nc
    res_regional=test
    res=test.drop(columns=['Province_State','Country_Region','Date','New Confirmed'])
    return res,res_regional 

In [89]:
# Call only when forcast and submission data are available
def get_countrywise_forcast_(target,country_name,state_name,num_days):
    temp=covid_timeseries[(covid_timeseries['Country/Region']==country_name)&(covid_timeseries['Province/State']==state_name)].query("ObservationDate>="+pivot_date)
    x_truth=temp.ObservationDate
    y_truth=temp[target]
    pred_=res_regional[(res_regional['Country_Region']==country_name) & ((res_regional['Province_State']==state_name))]
    x_pred=pred_.Date[0:num_days]
    y_pred=pred_[target][0:num_days]
    return list(x_truth),list(y_truth),list(x_pred),list(y_pred)

In [90]:
# Call only when forcast and submission data are available
def get_countrywise_forcast(country_name,state_name,num_days):
    temp=train[(train['Country/Region']==country_name)&(train['Province/State']==state_name)].query("ObservationDate>="+pivot_date)
    x_truth=temp.ObservationDate
    y_truth=temp.Confirmed
    pred_=res_regional[(res_regional['Country_Region']==country_name) & ((res_regional['Province_State']==state_name))]
    x_pred=pred_.Date[0:num_days]
    y_pred=pred_.ConfirmedCases[0:num_days]
    return list(x_truth),list(y_truth),list(x_pred),list(y_pred)

In [91]:
# Call only when forcast and submission data are available
def get_countrywise_forcast_Deaths(country_name,state_name,num_days):
    temp=train[(train['Country/Region']==country_name)&(train['Province/State']==state_name)].query("ObservationDate>="+pivot_date)
    x_truth=temp.ObservationDate
    y_truth=temp.Deaths
    pred_=res_regional[(res_regional['Country_Region']==country_name) & ((res_regional['Province_State']==state_name))]
    x_pred=pred_.Date[0:num_days]
    y_pred=pred_.Fatalities[0:num_days]
    return list(x_truth),list(y_truth),list(x_pred),list(y_pred)

In [92]:
# Utility Method to convert newly added prediction to cumulative [Not Accurate]
def get_cumulative_confirmed_cases(world_data_):
    world_data_=world_data_.sort_values(['Country_Region','Date'])
    temp=[0*i for i in range(len(world_data_))]
    world_data_['Cumulative Confirmed']=world_data_['New Confirmed']
    for i in tqdm(range(1,len(world_data_))):
        if(world_data_['Country_Region'].iloc[i]!=world_data_['Country_Region'].iloc[i-1]):
            world_data_['Cumulative Confirmed'].iloc[i]=world_data_['ConfirmedCases'].iloc[i]
    for i in tqdm(range(1,len(world_data_))):    
        if(world_data_['Country_Region'].iloc[i]==world_data_['Country_Region'].iloc[i-1]):
            world_data_['Cumulative Confirmed'].iloc[i]=world_data_['Cumulative Confirmed'].iloc[i]+world_data_['Cumulative Confirmed'].iloc[i-1]
    return world_data_

## Forcasting

In [93]:
# num_days = Num of days for which Forcasting is required
#forcast_start_date='2020-04-01'

res_confirmed=forcast(model_c,X_test_c,forcast_start_date,num_days=50)
res_deaths=forcast(model_d,X_test_d,forcast_start_date,num_days=50)
res_new_confirmed=forcast(model_nc,X_test_nc,forcast_start_date,num_days=50)

## Submission

In [94]:
# res_regional contains submission data along with extra columns
sub,res_regional=prepare_submission(res_confirmed,res_deaths,res_new_confirmed,test,pivot_date)
sub.to_csv('submission.csv',index=None)
sub.head()

,ForecastId,ConfirmedCases,Fatalities
7,8,237,4
8,9,304,3
9,10,388,3
10,11,489,3
11,12,612,3


### Countrywise Forcast

In [95]:
x_truth_Ge,y_truth_Ge,x_pred_Ge,y_pred_Ge=get_countrywise_forcast_('New Confirmed','Germany','NoState',7)
x_truth_In,y_truth_In,x_pred_In,y_pred_In=get_countrywise_forcast_('New Confirmed','India','NoState',7)
x_truth_Sp,y_truth_Sp,x_pred_Sp,y_pred_Sp=get_countrywise_forcast_('New Confirmed','Spain','NoState',7)
x_truth_It,y_truth_It,x_pred_It,y_pred_It=get_countrywise_forcast_('New Confirmed','Italy','NoState',7)

In [96]:
x_truth_Ge,y_truth_Ge,x_pred_Ge,y_pred_Ge=get_countrywise_forcast('Germany','NoState',7)
x_truth_In,y_truth_In,x_pred_In,y_pred_In=get_countrywise_forcast('India','NoState',7)
x_truth_Sp,y_truth_Sp,x_pred_Sp,y_pred_Sp=get_countrywise_forcast('Spain','NoState',7)
x_truth_It,y_truth_It,x_pred_It,y_pred_It=get_countrywise_forcast('Italy','NoState',7)

### Visulizing the Prediction

In [97]:
fig = make_subplots(rows=2, cols=2)
fig.update_layout(template='plotly_dark')
fig.add_trace(go.Scatter(x=x_truth_In, 
                         y=y_truth_In,
                         mode='lines+markers',
                         name='Actual_India',
                         line=dict(color='#CCFFCC', width=3)),1,1)
fig.add_trace(go.Scatter(x=x_pred_In, 
                         y=y_pred_In,
                         mode='lines+markers',
                         name='Predicted_India',
                         line=dict(color='red', width=1)),1,1)

fig.add_trace(go.Scatter(x=x_truth_Sp, 
                         y=y_truth_Sp,
                         mode='lines+markers',
                         name='Actual_Spain',
                         line=dict(color='yellow', width=3)),1,2)
fig.add_trace(go.Scatter(x=x_pred_Sp, 
                         y=y_pred_Sp,
                         mode='lines+markers',
                         name='Predicted_Spain',
                         line=dict(color='red', width=1)),1,2)

fig.add_trace(go.Scatter(x=x_truth_Ge, 
                         y=y_truth_Ge,
                         mode='lines+markers',
                         name='Actual_Germany',
                         line=dict(color='#E5CCFF', width=3)),2,1)
fig.add_trace(go.Scatter(x=x_pred_Ge, 
                         y=y_pred_Ge,
                         mode='lines+markers',
                         name='Predicted_Germany',
                         line=dict(color='red', width=1)),2,1)


fig.add_trace(go.Scatter(x=x_truth_It, 
                         y=y_truth_It,
                         mode='lines+markers',
                         name='Actual-Italy',
                         line=dict(color='#33FFFF', width=3)),2,2)
fig.add_trace(go.Scatter(x=x_pred_It, 
                         y=y_pred_It,
                         mode='lines+markers',
                         name='Predicted_Italy',
                         line=dict(color='red', width=1)),2,2)

fig.update_layout(template='plotly_dark',
                  title = 'COVID-19 Confirmed Cases prediction in India/Spain/Germany/Italy(27th March - 9th April)',
                  annotations=[
    ]
                 )

In [98]:
from math import sqrt 

mse_In = mean_squared_error(y_truth_In, y_pred_In)
rmse_In = sqrt(mse_In)
print("RMSE_IN %f" % rmse_In)

ValueError: Found input variables with inconsistent numbers of samples: [6, 7]

In [ ]:
x_truth_Ge,y_truth_Ge,x_pred_Ge,y_pred_Ge=get_countrywise_forcast_Deaths('Germany','NoState',7)
x_truth_In,y_truth_In,x_pred_In,y_pred_In=get_countrywise_forcast_Deaths('India','NoState',7)
x_truth_Sp,y_truth_Sp,x_pred_Sp,y_pred_Sp=get_countrywise_forcast_Deaths('Spain','NoState',7)
x_truth_It,y_truth_It,x_pred_It,y_pred_It=get_countrywise_forcast_Deaths('Italy','NoState',7)

In [ ]:
fig = make_subplots(rows=2, cols=2)
fig.update_layout(template='plotly_dark')
fig.add_trace(go.Scatter(x=x_truth_In, 
                         y=y_truth_In,
                         mode='lines+markers',
                         name='Actual_India',
                         line=dict(color='#CCFFCC', width=3)),1,1)
fig.add_trace(go.Scatter(x=x_pred_In, 
                         y=y_pred_In,
                         mode='lines+markers',
                         name='Predicted_India',
                         line=dict(color='red', width=1)),1,1)

fig.add_trace(go.Scatter(x=x_truth_Sp, 
                         y=y_truth_Sp,
                         mode='lines+markers',
                         name='Actual_Spain',
                         line=dict(color='yellow', width=3)),1,2)
fig.add_trace(go.Scatter(x=x_pred_Sp, 
                         y=y_pred_Sp,
                         mode='lines+markers',
                         name='Predicted_Spain',
                         line=dict(color='red', width=1)),1,2)

fig.add_trace(go.Scatter(x=x_truth_Ge, 
                         y=y_truth_Ge,
                         mode='lines+markers',
                         name='Actual_Germany',
                         line=dict(color='#E5CCFF', width=3)),2,1)
fig.add_trace(go.Scatter(x=x_pred_Ge, 
                         y=y_pred_Ge,
                         mode='lines+markers',
                         name='Predicted_Germany',
                         line=dict(color='red', width=1)),2,1)


fig.add_trace(go.Scatter(x=x_truth_It, 
                         y=y_truth_It,
                         mode='lines+markers',
                         name='Actual-Italy',
                         line=dict(color='#33FFFF', width=3)),2,2)
fig.add_trace(go.Scatter(x=x_pred_It, 
                         y=y_pred_It,
                         mode='lines+markers',
                         name='Predicted_Italy',
                         line=dict(color='red', width=1)),2,2)

fig.update_layout(template='plotly_dark',
                  title = 'COVID-19 Death Cases prediction in India/Spain/Germany/Italy(2nd April - 8th April)',
                  annotations=[
    ]
                 )

In [ ]:
from math import sqrt 

mse_In = mean_squared_error(y_truth_In, y_pred_In)
rmse_In = sqrt(mse_In)
print("RMSE_IN %f" % rmse_In)
